<a href="https://colab.research.google.com/github/soumik12345/point-cloud-segmentation/blob/inference/notebooks/train_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/soumik12345/point-cloud-segmentation

Cloning into 'point-cloud-segmentation'...
remote: Enumerating objects: 726, done.
remote: Counting objects: 100% (726/726), done.
remote: Compressing objects: 100% (456/456), done.
remote: Total 726 (delta 425), reused 483 (delta 224), pack-reused 0
Receiving objects: 100% (726/726), 2.98 MiB | 14.81 MiB/s, done.
Resolving deltas: 100% (425/425), done.


In [1]:
!pip install -qqq wandb ml_collections

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 88 kB 9.0 MB/s 
     |████████████████████████████████| 139 kB 73.8 MB/s 
     |████████████████████████████████| 180 kB 72.7 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


In [1]:
import sys

sys.path.append("point-cloud-segmentation")

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
import os

import wandb
import wandb.keras
from datetime import datetime

from tensorflow.keras import optimizers, callbacks
from tensorflow.keras import mixed_precision

from point_seg import TFRecordLoader, ShapeNetCoreLoaderInMemory
from point_seg import models, utils

In [4]:
#@title Configs
#@markdown Get your `wandb_api_key` from https://wandb.ai/authorize.
wandb_api_key = "8d7149fe07496df2aaab8e9856a6ed8564e2a644" #@param {type:"string"}
object_category = "Airplane" #@param {type:"string"}
num_points = 1024 #@param {type:"integer"}
batch_size = 32 #@param {type:"integer"}
epochs = 100 #@param {type:"integer"}
artifact_location = "gs://pointnet-segmentation" #@param {type:"string"}
initial_lr = 1e-3 #@param {type:"number"}
drop_every = 10 #@param {type:"integer"}
decay_factor = 0.5 #@param {type:"number"}

In [5]:
timestamp = datetime.utcnow().strftime("%y%m%d-%H%M%S")
strategy = utils.initialize_device()
batch_size = 32 * strategy.num_replicas_in_sync

INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.28.24.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.28.24.130:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [6]:
wandb.init(
    project='pointnet_shapenet_core',
    name=f"{object_category}_{timestamp}",
    entity="pointnet",
)

wandb: Currently logged in as: pointnet (use `wandb login --relogin` to force relogin)


In [7]:
tfrecord_loader = TFRecordLoader(
    tfrecord_dir=os.path.join(
        artifact_location, "tfrecords"
    ),
    object_category=object_category,
)
train_dataset, val_dataset = tfrecord_loader.get_datasets(
    batch_size=batch_size, drop_remainder=True
)

In [8]:
lr_scheduler = utils.StepDecay(initial_lr, drop_every, decay_factor)
lr_callback = callbacks.LearningRateScheduler(
    lambda epoch: lr_scheduler(epoch), verbose=True
)

# Tensorboard Callback
logs_dir = os.path.join(
    "logs", f"{object_category}_{timestamp}"
)
logs_dir = os.path.join(artifact_location, logs_dir)
tb_callback = callbacks.TensorBoard(log_dir=logs_dir)

# ModelCheckpoint Callback
checkpoint_path = os.path.join(
    artifact_location,
    "training_checkpoints",
    f"{object_category}_{timestamp}",
)
checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_best_only=True, save_weights_only=True,
)

callback_list = [
    tb_callback,
    checkpoint_callback,
    lr_callback,
    wandb.keras.WandbCallback()
]

In [9]:
with strategy.scope():
    optimizer = optimizers.Adam(learning_rate=initial_lr)
    _, y = next(iter(train_dataset))
    num_classes = y.shape[-1]
    model = models.get_shape_segmentation_model(num_points, num_classes)

model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)


In [10]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=callback_list,
)

Epoch 1/100

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
      6/Unknown - 26s 681ms/step - loss: 14.7031 - accuracy: 0.3650WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1049s vs `on_train_batch_end` time: 0.4812s). Check your callbacks.


11/11 [==============================] - 32s 924ms/step - loss: 11.4767 - accuracy: 0.4992 - val_loss: 8.8147 - val_accuracy: 0.6701
Epoch 2/100

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
11/11 [==============================] - 2s 185ms/step - loss: 5.5254 - accuracy: 0.7511 - val_loss: 9.3384 - val_accuracy: 0.7563
Epoch 3/100

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
11/11 [==============================] - 2s 176ms/step - loss: 4.8946 - accuracy: 0.7958 - val_loss: 6.8431 - val_accuracy: 0.7931
Epoch 4/100

Epoch 00004: LearningRateScheduler setting learning rate to 0.001.
11/11 [==============================] - 2s 177ms/step - loss: 4.7147 - accuracy: 0.8159 - val_loss: 6.4405 - val_accuracy: 0.8045
Epoch 5/100

Epoch 00005: LearningRateScheduler setting learning rate to 0.001.
11/11 [==============================] - 2s 211ms/step - loss: 4.6225 - accuracy: 0.8324 - val_loss: 6.1094 - val_accuracy: 0.8172
Epoch 6/100

Epoch 0000

In [11]:
serialization_path = os.path.join(
    artifact_location, "final_models",
    f"{object_category}_{timestamp}",
)
model.load_weights(checkpoint_path)
model.save(serialization_path)

INFO:tensorflow:Assets written to: gs://pointnet-segmentation/final_models/Airplane_211029-144027/assets


INFO:tensorflow:Assets written to: gs://pointnet-segmentation/final_models/Airplane_211029-144027/assets
